### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Find the number of sequences

In [ ]:
import pathlib
vir_path = pathlib.Path('./sequences/edwards/virus/')
host_path = pathlib.Path('./sequences/edwards/host/')
vir_count = 0
host_count = 0
for vir_file in vir_path.iterdir():
    vir_count += 1
for host_file in host_path.iterdir():
    host_count += 1

print(f'virus files: {vir_count}')
print(f'host files: {host_count}')
print(f'paired files: {vir_count * host_count}')

## Importing the dataset

### Merge the datasets

In [ ]:
import pathlib
from functools import reduce

features_path = pathlib.Path('./features/')
dfs = [pd.read_csv(feat_file, sep='\t', header=0) for feat_file in features_path.iterdir()]
final_df = reduce(lambda left,right: pd.merge(left,right,on=["#virus", "bacteria"],
                                                    how="outer"), dfs)
final_df.shape

### Replace the column names

In [ ]:
col_names = ['#virus', 'host']
col_names += [file.stem for file in features_path.iterdir()]
final_df.columns = col_names
final_df.head()

In [ ]:
check_rows = final_df[~final_df.isnull().any(axis=1)]
check_rows.head()

## Save the dataframe to files

In [ ]:
final_df.to_csv('main_df.csv',index=False)
final_df.to_pickle('./main_df.pkl')